In [1]:
import pandas as pd
import matplotlib as mpl

In [4]:
def read_csv(file_name):
    return pd.read_csv(file_name)

yad2_df = read_csv('raw_yad2_data_test.csv')
homeless_df = read_csv('raw_homeless_data_test.csv')

df = pd.concat([yad2_df, homeless_df], ignore_index=True, sort=False)

print(df.head)


<bound method NDFrame.head of    Post link          City   Neighborhood  Size  Number of rooms  \
0   ru8q1kyv        סביניה      מרכז העיר   105                4   
1   lc0eawvk         אשדוד        רובע ב'    68                3   
2   k5vqkej7          חיפה       הדר מרכז    45                1   
3   c4jltnfv   תל אביב יפו   נאות אפקה ב'    90                3   
4   t37185rk        רחובות   חצרות המושבה   108                4   
..       ...           ...            ...   ...              ...   
56    684958  יהוד מונוסון           מכבי    65                3   
57    684957          חיפה      נווה שאנן     1               35   
58    684956  יהוד מונוסון           מכבי    65                3   
59    684954    קרית טבעון          ותיקה   150               55   
60    684953     פתח תקווה     מחנה יהודה    90               35   

    Floor number   Taxes  Price  Room mates  Furniture  Elevator  \
0              2  1000.0   4000           1          1         1   
1              3 